# BRAZO ROBOTICO

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pybullet as p
import pybullet_data

class FlipCupPyBulletEnv(gym.Env):
    metadata = {'render_modes': ['human']}
    
    def __init__(self, render_mode=None):
        super().__init__()
        try:
            p.disconnect()
        except:
            pass
        self.render_mode = render_mode
        self.physicsClient = p.connect(p.GUI if render_mode == 'human' else p.DIRECT)
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        
        # Aumentamos a 4 acciones: controlamos articulaciones 1, 2, 3 y 4
        self.action_space = spaces.Box(low=-1, high=1, shape=(4,), dtype=np.float32)
        
        # Observación: posiciones y velocidades de juntas, posición y orientación del vaso, velocidad del vaso
        # 4 (juntas) + 4 (vel juntas) + 3 (pos vaso) + 4 (ori vaso) + 3 (vel vaso) = 18
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(18,), dtype=np.float32)
        
        # Parámetros ajustados
        self.joint_force = 400  # Más fuerza para movimientos más dinámicos
        self.joint_max_velocity = 1.0  # Más rápido
        self.sim_steps = 15  # 8 Más pasos para mayor estabilidad
        self.max_steps = 800  # 500 Límite de pasos por episodio
        self.step_counter = 0
        
        # Umbrales para recompensa
        self.dist_threshold = 0.05  # Distancia cercana al vaso
        self.flip_threshold = 0.7  # Umbral para considerar el vaso volteado
        self.height_threshold = 0.02  # Umbral para considerar el vaso en el suelo (tirado)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        p.resetSimulation()
        p.setGravity(0, 0, -9.8)
        p.loadURDF("plane.urdf")
        
        # Cargar robot
        self.robotId = p.loadURDF("kuka_iiwa/model.urdf", [0, 0, 0], useFixedBase=True)
        
        # Cargar vaso con fricción para evitar deslizamiento excesivo
        self.cup_x = np.random.uniform(0.5, 0.9)
        collision_shape = p.createCollisionShape(p.GEOM_CYLINDER, radius=0.03, height=0.09)
        visual_shape = p.createVisualShape(p.GEOM_CYLINDER, radius=0.03, length=0.09, rgbaColor=[0.8, 0.8, 0.2, 1])
        self.cupId = p.createMultiBody(
            baseMass=0.2,  # Aumentamos masa para estabilidad
            baseCollisionShapeIndex=collision_shape,
            baseVisualShapeIndex=visual_shape,
            basePosition=[self.cup_x, 0, 0.045]
        )
        p.changeDynamics(self.cupId, -1, lateralFriction=0.8, spinningFriction=0.1, rollingFriction=0.1)
        
        # Reiniciar contador de pasos
        self.step_counter = 0
        
        # Observación inicial
        obs = self._get_obs()
        return obs, {}

    def step(self, action):
        # Aplicar acciones a las articulaciones 1, 2, 3 y 4
        for i, joint in enumerate([1, 2, 3, 4]):
            p.setJointMotorControl2(
                self.robotId, joint, p.POSITION_CONTROL,
                targetPosition=float(action[i]), force=self.joint_force, maxVelocity=self.joint_max_velocity
            )
        
        # Simular
        for _ in range(self.sim_steps):
            p.stepSimulation()
        
        # Incrementar contador
        self.step_counter += 1
        
        # Obtener observación
        obs = self._get_obs()
        cup_pos, cup_ori = p.getBasePositionAndOrientation(self.cupId)
        cup_vel = p.getBaseVelocity(self.cupId)[0]  # Velocidad lineal
        ee_pos = p.getLinkState(self.robotId, 6)[0]
        
        # Calcular recompensa
        dist = np.linalg.norm(np.array(ee_pos) - np.array(cup_pos))
        reward = -0.1 * dist  # Penalizar distancia (escalada)
        
        # Bonus por acercarse mucho
        if dist < self.dist_threshold:
            reward += 2.0
        
        # Bonus por voltear el vaso (basado en orientación)
        if abs(cup_ori[0]) > self.flip_threshold or abs(cup_ori[1]) > self.flip_threshold:
            reward += 20.0  # Gran recompensa por voltear
        
        # Bonus por tirar el vaso (en el suelo)
        if cup_pos[2] < self.height_threshold:
            reward += 30.0  # Mayor recompensa por tirar
        
        # Penalización por pasos largos
        reward -= 0.01
        
        # Condiciones de terminación
        terminated = (
            (abs(cup_ori[0]) > self.flip_threshold or abs(cup_ori[1]) > self.flip_threshold) or
            (cup_pos[2] < self.height_threshold)
        )
        truncated = self.step_counter >= self.max_steps
        
        info = {'distance': dist}
        return obs, reward, terminated, truncated, info

    def render(self):
        if self.render_mode == 'human':
            p.configureDebugVisualizer(p.COV_ENABLE_RENDERING, 1)

    def close(self):
        p.disconnect()

    def _get_obs(self):
        # Posiciones y velocidades de las juntas 1, 2, 3, 4
        joint_states = p.getJointStates(self.robotId, [1, 2, 3, 4])
        joint_pos = np.array([s[0] for s in joint_states])
        joint_vel = np.array([s[1] for s in joint_states])
        
        # Posición, orientación y velocidad del vaso
        cup_pos, cup_ori = p.getBasePositionAndOrientation(self.cupId)
        cup_vel = p.getBaseVelocity(self.cupId)[0]  # Velocidad lineal
        
        # Concatenar observación
        obs = np.concatenate([joint_pos, joint_vel, cup_pos, cup_ori, cup_vel])
        return obs



In [ ]:
# --- Entrenamiento PPO ---
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CheckpointCallback

# Guardar modelos periódicamente
checkpoint_callback = CheckpointCallback(save_freq=10000, save_path='./logs/', name_prefix='ppo_flipcup')

env = FlipCupPyBulletEnv(render_mode='human')
model = PPO(
    "MlpPolicy", env,
    verbose=1,
    device="cuda",
    learning_rate= 3e-4, #3e-4,  # Aumentada ligeramente
    n_steps=2048,
    batch_size=64,  # Añadido para estabilidad
    gamma=0.99,  # Aumentado para valorar recompensas futuras
    ent_coef=0.01,
    clip_range=0.4,  # Aumentado para mayor exploración
)
model.learn(total_timesteps=150000, callback=checkpoint_callback)  # Más pasos
model.save("ppo_flipcup_pybullet_v3")
env.close()



In [ ]:
# --- Prueba visual ---
import time
env = FlipCupPyBulletEnv(render_mode='human')
model = PPO.load("ppo_flipcup_pybullet_v3", env=env)
obs, _ = env.reset()
done = False
while not done:
    action, _ = model.predict(obs)
    obs, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    time.sleep(0.04)  # Más fluido
env.close()